In [1]:
# install dependencies
! pip install openai psycopg2 pandas wget python-dotenv panel datasets jupyter_bokeh

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import io
from openai import OpenAI
import psycopg2
from dotenv import load_dotenv
import json
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import zipfile
import os
import re
import tempfile
import openai

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import HTML, display
display(HTML("<style>div.output_scroll { height: 1000px; }</style>"))

/Users/yanyan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

if os.getenv("OPENAI_API_KEY") is not None:
    print("Your OPENAI_API_KEY is ready")
else:
    print("Your OPENAI_API_KEY environment variable was not found")

Your OPENAI_API_KEY is ready


In [45]:
load_dotenv()
connection_string = os.getenv('DATABASE_URL')
if not connection_string:
     raise ValueError("Please provide a valid connection string either in the code or in the .env file as DATABASE_URL.")

# Connect using the connection string
connection = psycopg2.connect(connection_string)

# Create a new cursor object
cursor = connection.cursor()

# Execute this query to test the database connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Your database connection was successful!")
else:
    print("Your connection failed.")

True

Your database connection was successful!


In [5]:
# Only recompute embeddings when there are structural changes to the data!
recompute_cocktail_embeddings = False

# Build embeddings using HuggingFace's cocktails_recipe_no_brand dataset
if recompute_cocktail_embeddings:

    cocktails_raw = load_dataset("erwanlc/cocktails_recipe_no_brand")

    def generate_embeddings(text):
        response = client.embeddings.create(
            input=text,
            model="text-embedding-ada-002",
        )
        return response.data[0].embedding

    def is_non_empty_string(field):
        return field and field.strip()

    def concatenate_fields(row):
        return ' '.join(f"{field}: {row[field]}." for field in required_fields)

    required_fields = ['title', 'glass', 'garnish', 'recipe', 'ingredients']
    cocktails = [cocktail for cocktail in cocktails_raw['train'] if all(is_non_empty_string(cocktail[field]) for field in required_fields)]
    df_cocktails = pd.DataFrame(cocktails)

    print(f"The number of cocktail recipes available: {len(df_cocktails)}")

    tqdm.pandas(desc="Generating combined embeddings")
    df_cocktails['combined_text'] = df_cocktails.apply(concatenate_fields, axis=1)
    df_cocktails['combined_embedding'] = df_cocktails['combined_text'].progress_apply(generate_embeddings)


In [6]:
# # rollback an aborted transaction when the previous transaction is errored out
# try:
#     cursor.execute("ROLLBACK")
#     connection.commit()
# except Exception as e:
#     print("Error rolling back transaction:", e)

In [7]:
# create recipe embedding table
create_table_sql = '''
DROP TABLE IF EXISTS public.cocktails;

CREATE TABLE IF NOT EXISTS public.cocktails (
    id SERIAL PRIMARY KEY,
    title TEXT,
    glass TEXT,
    garnish TEXT,
    recipe TEXT,
    ingredients TEXT,
    combined_embedding vector(1536)
);
'''

# SQL statement for creating indexes
create_indexes_sql = '''
CREATE INDEX ON public.cocktails USING ivfflat (combined_embedding) WITH (lists = 100);
'''

# Execute the SQL statements
cursor.execute(create_table_sql)
cursor.execute(create_indexes_sql)

# Commit the changes
connection.commit()


In [8]:
# extract the pre-computed embedding data
current_directory = os.getcwd()
zip_file_path = os.path.join(current_directory, "data/cocktail_recipes_6k_with_embedding.zip")
output_directory = os.path.join(current_directory, "data")

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_directory)

file_name = 'cocktail_recipes_500_embedded.csv'
file_path = os.path.join(output_directory, file_name)
if os.path.exists(file_path):
    print(f"The csv file {file_name} exists in the data directory.")
else:
    print(f"The csv file {file_name} does not exist in the data directory.")

The csv file cocktail_recipes_500_embedded.csv exists in the data directory.


In [9]:
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield line

# Create a StringIO object to store the modified lines
modified_lines = io.StringIO(''.join(list(process_file(file_path))))

# Create the COPY command for copy_expert
copy_command = '''
COPY public.cocktails (
    title,
    glass,
    garnish,
    ingredients,
    recipe,
    combined_embedding)
FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ',');
'''

# Execute the COPY command using copy_expert
cursor.copy_expert(copy_command, modified_lines)

# Commit the changes
connection.commit()


In [10]:
def query_neon(query,top_k=3):
    # Create an embedding vector from the user query
    embedded_query = client.embeddings.create(
        input=query,
        model="text-embedding-ada-002",
    ).data[0].embedding

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create the SQL query
    query_sql = f"""
    SELECT
        id,
        title,
        glass,
        garnish,
        ingredients,
        recipe,
        l2_distance(combined_embedding,'{embedded_query_pg}'::VECTOR(1536)) AS distance_score
    FROM cocktails
    ORDER BY combined_embedding <-> '{embedded_query_pg}'::VECTOR(1536)
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results


In [11]:
def format_matched_recipes(query_results):
    formatted_recipes = []

    for i, recipe in enumerate(query_results):
        recipe_string = (
            f"Recipe {i + 1}: {recipe[1]}\n"
            f"- Recipe ID: {recipe[0]}\n"
            f"- Similarity Score: {1 - recipe[6]:.3f}\n"
            f"- Glass: {recipe[2]}\n"
            f"- Garnish: {recipe[3]}\n"
            f"- Ingredients: {recipe[4]}\n"
            f"- Recipe: {recipe[5]}\n"
        )
        formatted_recipes.append(recipe_string)

    formatted_results = "\n\n".join(formatted_recipes)
    return formatted_results


In [12]:
def extract_user_input(user_input):
    functions = [
    {
    "name": "extract_cocktail_attributes",
    "description": "extract the cocktail attributes the user is looking to create",
    "parameters":
        {
            "type": "object",
            "properties": {
                "title":
                    {
                        "type": "string",
                        "description": "the name of the cocktail"
                    },
                "glass":
                    {
                        "type": "string",
                        "description": "the type of glass to serve the cocktail"
                    },
                "garnish":
                    {
                        "type": "string",
                        "description": "the instruction on how and what to use to garnish the cocktail"
                    },
                "recipe":
                    {
                        "type": "string",
                        "description": "the instruction on how to make the cocktail"
                    },
               "ingredients": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "ingredient_name": {
                                    "type": "string",
                                    "description": "the name of the ingredient"
                                },
                                "quantity": {
                                    "type": "string",
                                    "description": "the quantity of the ingredient (optional)"
                                }
                            }
                        },
                        "description": "the list of ingredients to use to make the cocktail"
                    }
            },
            "required": ["title", "glass", "garnish", "recipe", "ingredients"]
        }
    }]

    response = client.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "system", "content": "You are a knowledgeable bartender assistant."},
                {"role": "user", "content": user_input}
            ],
            functions=functions,
            temperature=0.2
        )

    user_input = response.choices[0].message.function_call.arguments
    user_input_json = json.loads(user_input)
    return user_input_json

def formated_query_input(user_input_json):
    formatted_query = ""
    for key, value in user_input_json.items():
        if key == 'ingredients':
            formatted_query += f"{key}: ["
            for ingredient in value:
                formatted_query += f"[{ingredient['quantity']}, '{ingredient['ingredient_name']}'], "
            formatted_query = formatted_query.rstrip(", ") + "]. "
        else:
            formatted_query += f"{key}: {value}. "
    return formatted_query


In [13]:
def match_cocktails(user_input, verbose_ouput=False):
    user_input_json = extract_user_input(user_input)

    formatted_query = formated_query_input(user_input_json)
    query_results = query_neon(formatted_query)
    formatted_query_results = format_matched_recipes(query_results)

    if verbose_ouput:
        print(f"Cocktail attributes extracted from user input:")
        print(json.dumps(user_input_json, indent=4))
        print()
        print(f"Matched cocktail recipes:")
        print(formatted_query_results)

    return formatted_query_results


In [14]:
user_input_example = "I want an old fashion served in a whiskey glass and I want to have some minty garnish, use bourbon. I'd like to serve this during Thanksgiving, so some cranberry or apple liqueur will make it better."
matched_cocktails = match_cocktails(user_input_example, verbose_ouput=True)

Cocktail attributes from user input:
{
    "title": "Old Fashion",
    "glass": "whiskey glass",
    "garnish": "minty garnish",
    "recipe": "1. In a mixing glass, muddle the sugar cube and bitters.\n2. Add bourbon and stir until well mixed.\n3. Strain into a whiskey glass filled with ice.\n4. Garnish with a mint sprig.",
    "ingredients": [
        {
            "ingredient_name": "sugar cube",
            "quantity": "1"
        },
        {
            "ingredient_name": "bitters",
            "quantity": "2 dashes"
        },
        {
            "ingredient_name": "bourbon",
            "quantity": "2 oz"
        },
        {
            "ingredient_name": "mint sprig",
            "quantity": "1"
        }
    ]
}

Matched cocktail recipes:
Recipe 1: Old Fashioned
- Recipe ID: 4276
- Similarity Score: 0.655
- Glass: Old-fashioned glass
- Garnish: Orange zest twist
- Ingredients: [['6 cl', 'Bourbon whiskey'], ['1.25 cl', 'Sugar syrup (65.0°brix, 2 sugar to 1 water rich syrup)'

In [19]:
user_input_example_2 = "Give me a drink to celebrate new year; Champagne based; easy recipe; not too many ingredients"
matched_cocktails = match_cocktails(user_input_example_2, verbose_ouput=True)

Cocktail attributes from user input:
{
    "title": "New Year's Champagne Cocktail",
    "glass": "Champagne flute",
    "garnish": "Lemon twist",
    "recipe": "1. Add 1 sugar cube to the bottom of a Champagne flute.\n2. Add a few dashes of Angostura bitters.\n3. Fill the glass with chilled Champagne.\n4. Garnish with a lemon twist.",
    "ingredients": [
        {
            "ingredient_name": "Sugar cube",
            "quantity": "1"
        },
        {
            "ingredient_name": "Angostura bitters",
            "quantity": "few dashes"
        },
        {
            "ingredient_name": "Champagne",
            "quantity": "chilled"
        },
        {
            "ingredient_name": "Lemon twist",
            "quantity": "1"
        }
    ]
}

Matched cocktail recipes:
Recipe 1: Champagne Cocktail
- Recipe ID: 323
- Similarity Score: 0.606
- Glass: Flute glass
- Garnish: Orange zest twist (discarded)
- Ingredients: [['1 cube', 'Sugar cube'], ['3 dash', 'Angostura Aromatic Bi

In [76]:
import panel as pn
pn.extension()

# Create a Panel app
app = pn.Column()

# Create a TextInput widget for user input
input_widget = pn.widgets.TextAreaInput(value="", width=400, height=100)

# Create a function to handle user input and display results
def on_submit(event):
    user_input = input_widget.value
    matched_cocktails_ui = match_cocktails(user_input)
    results_widget.object = f"<pre>{matched_cocktails}</pre>"  # Update results_widget

# Create a Button widget to submit user input
submit_button = pn.widgets.Button(name="Match Cocktails")
submit_button.on_click(on_submit)

# Create an HTML widget to display results with text wrapping
results_widget = pn.pane.HTML("", width=1000, height=600, style={"overflow": "auto"})

# Add widgets to the app
app.append("### Cocktail Matcher")
app.append("Enter your cocktail preferences:")
app.append(input_widget)
app.append(submit_button)
app.append(results_widget)

app.servable()


Watcher(inst=Button(name='Match Cocktails'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_submit at 0x17aeeb310>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

/var/folders/4p/rm97zzdx39x_653ljcypwklh0000gn/T/ipykernel_36078/2221696623.py:21: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  results_widget = pn.pane.HTML("", width=1000, height=600, style={"overflow": "auto"})


BokehModel(combine_events=True, render_bundle={'docs_json': {'1319297a-6584-4db8-810c-6b4e3cf635c8': {'version…